In [1]:
import os
import shutil
import tempfile
import time
from typing import Any, Dict, List, Tuple, Union

import argparse
import joblib
from joblib import Parallel, delayed
import numpy as np
import pandas as pd 
import pickle as pkl
from termcolor import cprint
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm 
import wandb 

from logger import Logger
import utils.baseline_config as config
import utils.baseline_utils as baseline_utils
from utils.lstm_utils import ModelUtils, LSTMDataset
from prog_utils import * 
from models import * 
from lstm_train_test import * 

use_cuda = torch.cuda.is_available()
if use_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
global_step = 0
best_loss = float("inf")
np.random.seed(100)

ROLLOUT_LENS = [1, 10, 30]

In [5]:
import sys
sys.argv = ['']
args = parse_arguments()
args.use_delta = True

data_dict = baseline_utils.get_data(args, 'none')


Loading Test data ...
Creating relative distances...
Test Size: 39472
Loading Train data ...
Creating relative distances...
Train Size: 205942
Loading Val data ...
Creating relative distances...
Val Size: 39472


In [2]:
baseline_name = 'val_prep_baseline.pkl'
baseline_val_pred = pd.read_pickle(f'Traj/{baseline_name}')

In [16]:
from utils.baseline_config import FEATURE_FORMAT
def get_xy(feature_entry):
    xy_traj = feature_entry[:, [FEATURE_FORMAT['X'], FEATURE_FORMAT['Y']]].astype("float")
    return xy_traj

def get_min_ade(pred_trajs, gt_xy):
    min_ade = 1000000
    min_idx = -1
    for (i, pred_traj) in enumerate(pred_trajs):
        ade = np.mean(np.linalg.norm(pred_traj - gt_xy, axis=1))
        if ade < min_ade:
            min_ade = ade
            min_idx = i

    return min_ade, min_idx

def get_min_fde(pred_trajs, gt_xy):
    fdes = [np.linalg.norm(t[-1] - gt_xy[-1]) for t in pred_trajs]
    min_fde = min(fdes)
    min_idx = fdes.index(min_fde)

    return min_fde, min_idx

In [27]:
min_ades = []
min_fdes = [] 

val_helpers = data_dict['val_helpers']
total_num = val_helpers.shape[0]
for i in tqdm(range(total_num)):
    example = val_helpers.iloc[i]
    xy_traj = get_xy(example.FEATURES)
    obs_xy = xy_traj[:args.obs_len, :]
    gt_xy = xy_traj[args.obs_len:, :]
    fitted_xy = baseline_val_pred[example['SEQUENCE']]

    min_ades.append(get_min_ade(fitted_xy, gt_xy)[0])
    min_fdes.append(get_min_fde(fitted_xy, gt_xy)[0])

avg_ade = np.mean(min_ades)
avg_fde = np.mean(min_fdes)
print("Average min ade: {}".format(avg_ade))
print("Average min fde: {}".format(avg_fde))

100%|██████████| 39472/39472 [00:13<00:00, 2882.85it/s]

Average min ade: 2.0643491062589763
Average min fde: 4.591912308782257
